proj-housing-insecurity

Task 8: Using the filtered datasets created in task 4, compare 2014 and 2019 ACS data for Orange County, FL. Which variables changed significantly? Which stayed more or less the same? Are any of these changes meaningful even after taking margins of error into account?

In [1]:

# dataframe

import pandas as pd
from scipy import stats 
import seaborn as sns #plot (if needed)
import matplotlib.pyplot as plt #plot (if needed)
import os #set working directory
import itertools as iter #run iterations on a df
#extend borders 
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
#Define function that will be used to combine lists of variables and remove lists that are not contained in both 
def intersection(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    return lst3

In [3]:
#set wd
#os.chdir("/Users/jordan/Documents/GitHub/datakind_housing_insecurity_2021-copy/American_Community_Survey_Data")

os.chdir("/Users/jordancreenaune/Desktop/Python_for_Data_Science/DataKind/September/American_Community_Survey_Data")
#import data 
orangec2014 = pd.read_csv("orange_acs5-2014_census.csv")
orangec2014 = orangec2014.T.drop_duplicates().T #remove duplicates columns  
orangec2019 = pd.read_csv("orange_acs5-2019_census.csv")
acs_dict = pd.read_csv("acs5_variable_dict_2014_2019.csv")
orangec2019 = orangec2019.T.drop_duplicates().T #remove duplicates columns 
#acs_dict_transformed = pd.read_csv("ACS_data_dict_labels.csv")



In [4]:
#rename each column to label for 2014 and 2019 

labeled_codes_2014 = intersection(orangec2014.columns, list(acs_dict[~acs_dict.label.isna()]['variable_code']))
labeled_codes_2014_df = orangec2014[['index']+labeled_codes_2014]


labeled_columns_2014_df = labeled_codes_2014_df
labeled_columns_2014_df.columns = ['index'] + [acs_dict[(acs_dict["variable_code"]==x)&(acs_dict["acs_year"]==2014)]["label"].values[0] for x in labeled_columns_2014_df.columns[1:]]



labeled_codes_2019 = intersection(orangec2019.columns, list(acs_dict[~acs_dict.label.isna()]['variable_code']))
labeled_codes_2019_df = orangec2019[['index']+labeled_codes_2019]
labeled_codes_2019_df = labeled_codes_2019_df.loc[:,~labeled_codes_2019_df.columns.duplicated()]

labeled_columns_2019_df = labeled_codes_2019_df
labeled_columns_2019_df.columns = ['index'] + [acs_dict[(acs_dict["variable_code"]==x)&(acs_dict["acs_year"]==2019)]["label"].values[0] for x in labeled_columns_2019_df.columns[1:]]

labeled_columns_2014_df.shape, labeled_columns_2019_df.shape



((207, 849), (207, 875))

In [5]:

cols_in_both = intersection(labeled_columns_2014_df.columns, labeled_columns_2019_df.columns)
len(cols_in_both)

524

In [6]:
#Remove columns in 2019 that do not match 

labeled_columns_2019_df = labeled_columns_2019_df[cols_in_both]
labeled_columns_2014_df = labeled_columns_2014_df[cols_in_both]

In [7]:
#Check number of columns in each df 
len(labeled_columns_2014_df.columns)


524

In [8]:
len(labeled_columns_2019_df.columns)


524

In [25]:
#Next task - order the census tract numbers so that theyre in order for both dfs - then do correlation matrix between both dfs 
#Sort ascending order then correlation matrix 
sort2019 = labeled_columns_2019_df.sort_values(by=['index'])
sort2014 = labeled_columns_2014_df.sort_values(by=['index'])

#duplicate these dfs to use later without the index 
sort2019i = sort2019
sort2014i = sort2014

In [26]:
#Make column 1 the index column so that it can be ignored in correlation matrix 
sort2019 = sort2019.set_index('index')
sort2014 = sort2014.set_index('index')

In [27]:
len(sort2019.columns)

523

In [12]:
#Check whether columns match - print head 

sort2014

,Estimate!!HOUSEHOLDS BY TYPE!!Total households,Estimate!!RELATIONSHIP!!Population in households,Percent!!RELATIONSHIP!!Population in households!!Householder,Estimate!!RELATIONSHIP!!Population in households!!Spouse,Percent!!RELATIONSHIP!!Population in households!!Spouse,Estimate!!RELATIONSHIP!!Population in households!!Child,Percent!!RELATIONSHIP!!Population in households!!Child,Estimate!!RELATIONSHIP!!Population in households!!Other relatives,Percent!!RELATIONSHIP!!Population in households!!Other relatives,Estimate!!MARITAL STATUS!!Males 15 years and over,...,Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Not Hispanic or Latino!!Native Hawaiian and Other Pacific Islander alone,Percent!!HISPANIC OR LATINO AND RACE!!Total population!!Not Hispanic or Latino!!Native Hawaiian and Other Pacific Islander alone,Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Not Hispanic or Latino!!Some other race alone,Percent!!HISPANIC OR LATINO AND RACE!!Total population!!Not Hispanic or Latino!!Some other race alone,Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Not Hispanic or Latino!!Two or more races,Percent!!HISPANIC OR LATINO AND RACE!!Total population!!Not Hispanic or Latino!!Two or more races,Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Not Hispanic or Latino!!Two or more races!!Two races including Some other race,Percent!!HISPANIC OR LATINO AND RACE!!Total population!!Not Hispanic or Latino!!Two or more races!!Two races including Some other race,"Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Not Hispanic or Latino!!Two or more races!!Two races excluding Some other race, and Three or more races","Percent!!HISPANIC OR LATINO AND RACE!!Total population!!Not Hispanic or Latino!!Two or more races!!Two races excluding Some other race, and Three or more races"
index,,,,,,,,,,,,,,,,,,,,,
"Census Tract 102, Orange County, Florida: Summary level: 140, state:12> county:095> tract:010200",2954,4694,62.9,658,14.0,161,3.4,113,2.4,2629,...,74,1.5,0,0.0,72,1.5,38,0.8,34,0.7
"Census Tract 103, Orange County, Florida: Summary level: 140, state:12> county:095> tract:010300",1734,2492,69.6,202,8.1,277,11.1,31,1.2,1100,...,0,0.0,0,0.0,71,2.7,0,0.0,71,2.7
"Census Tract 104, Orange County, Florida: Summary level: 140, state:12> county:095> tract:010400",483,1597,30.2,50,3.1,637,39.9,325,20.4,390,...,0,0.0,0,0.0,20,1.3,0,0.0,20,1.3
"Census Tract 105, Orange County, Florida: Summary level: 140, state:12> county:095> tract:010500",476,1097,43.4,28,2.6,246,22.4,220,20.1,407,...,0,0.0,0,0.0,48,4.4,0,0.0,48,4.4
"Census Tract 108.02, Orange County, Florida: Summary level: 140, state:12> county:095> tract:010802",1603,2777,57.7,327,11.8,428,15.4,101,3.6,1252,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Census Tract 185, Orange County, Florida: Summary level: 140, state:12> county:095> tract:018500",1464,3630,40.3,248,6.8,820,22.6,526,14.5,1619,...,0,0.0,6,0.1,87,2.1,0,0.0,87,2.1
"Census Tract 187, Orange County, Florida: Summary level: 140, state:12> county:095> tract:018700",1893,5136,36.9,703,13.7,1960,38.2,348,6.8,1515,...,0,0.0,0,0.0,21,0.4,0,0.0,21,0.4
"Census Tract 188, Orange County, Florida: Summary level: 140, state:12> county:095> tract:018800",1637,3111,52.6,376,12.1,438,14.1,122,3.9,1316,...,0,0.0,13,0.4,53,1.7,22,0.7,31,1.0


In [13]:
#Check 2019 df if it is matched with 2014 

sort2019

,Estimate!!HOUSEHOLDS BY TYPE!!Total households,Estimate!!RELATIONSHIP!!Population in households,Percent!!RELATIONSHIP!!Population in households!!Householder,Estimate!!RELATIONSHIP!!Population in households!!Spouse,Percent!!RELATIONSHIP!!Population in households!!Spouse,Estimate!!RELATIONSHIP!!Population in households!!Child,Percent!!RELATIONSHIP!!Population in households!!Child,Estimate!!RELATIONSHIP!!Population in households!!Other relatives,Percent!!RELATIONSHIP!!Population in households!!Other relatives,Estimate!!MARITAL STATUS!!Males 15 years and over,...,Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Not Hispanic or Latino!!Native Hawaiian and Other Pacific Islander alone,Percent!!HISPANIC OR LATINO AND RACE!!Total population!!Not Hispanic or Latino!!Native Hawaiian and Other Pacific Islander alone,Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Not Hispanic or Latino!!Some other race alone,Percent!!HISPANIC OR LATINO AND RACE!!Total population!!Not Hispanic or Latino!!Some other race alone,Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Not Hispanic or Latino!!Two or more races,Percent!!HISPANIC OR LATINO AND RACE!!Total population!!Not Hispanic or Latino!!Two or more races,Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Not Hispanic or Latino!!Two or more races!!Two races including Some other race,Percent!!HISPANIC OR LATINO AND RACE!!Total population!!Not Hispanic or Latino!!Two or more races!!Two races including Some other race,"Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Not Hispanic or Latino!!Two or more races!!Two races excluding Some other race, and Three or more races","Percent!!HISPANIC OR LATINO AND RACE!!Total population!!Not Hispanic or Latino!!Two or more races!!Two races excluding Some other race, and Three or more races"
index,,,,,,,,,,,,,,,,,,,,,
"Census Tract 102, Orange County, Florida: Summary level: 140, state:12> county:095> tract:010200",2838,4935,57.5,839,17.0,503,10.2,38,0.8,2629,...,0,0.0,21,0.4,55,1.1,0,0.0,55,1.1
"Census Tract 103, Orange County, Florida: Summary level: 140, state:12> county:095> tract:010300",1539,2279,67.5,193,8.5,166,7.3,20,0.9,1002,...,0,0.0,0,0.0,49,2.0,0,0.0,49,2.0
"Census Tract 104, Orange County, Florida: Summary level: 140, state:12> county:095> tract:010400",461,1127,40.9,73,6.5,424,37.6,107,9.5,394,...,0,0.0,0,0.0,26,2.3,0,0.0,26,2.3
"Census Tract 105, Orange County, Florida: Summary level: 140, state:12> county:095> tract:010500",493,1218,40.5,39,3.2,390,32.0,132,10.8,395,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
"Census Tract 108.02, Orange County, Florida: Summary level: 140, state:12> county:095> tract:010802",1482,3294,45.0,571,17.3,552,16.8,74,2.2,1538,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Census Tract 185, Orange County, Florida: Summary level: 140, state:12> county:095> tract:018500",1092,2864,38.1,274,9.6,820,28.6,271,9.5,1455,...,0,0.0,49,1.5,124,3.8,0,0.0,124,3.8
"Census Tract 187, Orange County, Florida: Summary level: 140, state:12> county:095> tract:018700",1810,5381,33.6,469,8.7,2299,42.7,502,9.3,1829,...,0,0.0,0,0.0,62,1.1,0,0.0,62,1.1
"Census Tract 188, Orange County, Florida: Summary level: 140, state:12> county:095> tract:018800",2377,4577,51.9,447,9.8,482,10.5,193,4.2,2353,...,0,0.0,12,0.3,77,1.7,0,0.0,77,1.7


In [14]:
len(sort2014.index) #Check number of rows within sort2014

207

In [15]:
len(sort2019.index) #Check number of rows within sort2019

207

We now have two data frames with identical column name attributes and row numbers. We can run correlation matrix 

In [16]:
#Check column names values are the same 
cols2019 = sort2019.columns
cols2014 = sort2014.columns 

common_cols = cols2019.intersection(cols2014)
cols_not_match = cols2019.difference(cols2014)
#cols_not_match
common_cols

Index(['Estimate!!HOUSEHOLDS BY TYPE!!Total households',
       'Estimate!!RELATIONSHIP!!Population in households',
       'Percent!!RELATIONSHIP!!Population in households!!Householder',
       'Estimate!!RELATIONSHIP!!Population in households!!Spouse',
       'Percent!!RELATIONSHIP!!Population in households!!Spouse',
       'Estimate!!RELATIONSHIP!!Population in households!!Child',
       'Percent!!RELATIONSHIP!!Population in households!!Child',
       'Estimate!!RELATIONSHIP!!Population in households!!Other relatives',
       'Percent!!RELATIONSHIP!!Population in households!!Other relatives',
       'Estimate!!MARITAL STATUS!!Males 15 years and over',
       ...
       'Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Not Hispanic or Latino!!Native Hawaiian and Other Pacific Islander alone',
       'Percent!!HISPANIC OR LATINO AND RACE!!Total population!!Not Hispanic or Latino!!Native Hawaiian and Other Pacific Islander alone',
       'Estimate!!HISPANIC OR LATINO AND RACE!!T

In [17]:
#Convert all values to ints 

sort2019 = sort2019.astype(int)
sort2014 = sort2014.astype(int)


sort2019.dtypes

Estimate!!HOUSEHOLDS BY TYPE!!Total households                                                                                                                      int64
Estimate!!RELATIONSHIP!!Population in households                                                                                                                    int64
Percent!!RELATIONSHIP!!Population in households!!Householder                                                                                                        int64
Estimate!!RELATIONSHIP!!Population in households!!Spouse                                                                                                            int64
Percent!!RELATIONSHIP!!Population in households!!Spouse                                                                                                             int64
                                                                                                                                                      

In [18]:
#Run correlation matrix 

corr_matrix= sort2014.corrwith(sort2019, axis=0)

corr_matrix.dropna(axis=0, inplace=True)

corr_matrix[corr_matrix>.1]

sort_mat = corr_matrix.sort_values()

In [21]:
sort_mat.head(10)

Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Not Hispanic or Latino!!American Indian and Alaska Native alone                         -0.057302
Estimate!!CLASS OF WORKER!!Civilian employed population 16 years and over!!Unpaid family workers                                                 -0.034690
Estimate!!HOUSE HEATING FUEL!!Occupied housing units!!Other fuel                                                                                 -0.017607
Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Not Hispanic or Latino!!Native Hawaiian and Other Pacific Islander alone                -0.013846
Estimate!!HOUSE HEATING FUEL!!Occupied housing units!!Coal or coke                                                                               -0.006747
Estimate!!SELECTED CHARACTERISTICS!!Occupied housing units!!Lacking complete plumbing facilities                                                  0.008021
Estimate!!HOUSE HEATING FUEL!!Occupied housing units!!Wood            

This correlation has been completed to demonstrate the differences between 2014 and 2019 Orange county florida - ACS data. First presented in the negative correlation, that has variables that have shown a significant decrease within the 5 years. Top 10 negative correlation are listed below. The majority of these are negative correlations for estimates relating to race, workers, heating types and ancestry 

In [22]:
sort_mat.tail(10)

Percent!!HOUSE HEATING FUEL!!Occupied housing units!!Bottled, tank, or LP gas                                                       1.0
Percent!!ANCESTRY!!Total population!!Slovak                                                                                         1.0
Percent!!HOUSE HEATING FUEL!!Occupied housing units!!Coal or coke                                                                   1.0
Percent!!HOUSE HEATING FUEL!!Occupied housing units!!Wood                                                                           1.0
Percent!!UNITS IN STRUCTURE!!Total housing units!!Boat, RV, van, etc.                                                               1.0
Percent!!HISPANIC OR LATINO AND RACE!!Total population!!Not Hispanic or Latino!!Native Hawaiian and Other Pacific Islander alone    1.0
Percent!!ANCESTRY!!Total population!!Czech                                                                                          1.0
Percent!!EMPLOYMENT STATUS!!Population 16 years 

Below is a new method to look at different ways in which these dfs are compared - create one side by side df where both years are presented under one variable with the intention to compare differences easier.  

In [28]:
#Concat dfs so that we can see side by side, df now has 2014 data then 2015 data horizontal 
df_all = pd.concat([sort2014i.set_index('index'), sort2019i.set_index('index')], 
                   axis='columns', keys=['2014', '2019'])
df_all



2014  \
                                                   Estimate!!HOUSEHOLDS BY TYPE!!Total households   
index                                                                                               
Census Tract 102, Orange County, Florida: Summa...                                           2954   
Census Tract 103, Orange County, Florida: Summa...                                           1734   
Census Tract 104, Orange County, Florida: Summa...                                            483   
Census Tract 105, Orange County, Florida: Summa...                                            476   
Census Tract 108.02, Orange County, Florida: Su...                                           1603   
...                                                                                           ...   
Census Tract 185, Orange County, Florida: Summa...                                           1464   
Census Tract 187, Orange County, Florida: Summa...                                           1893   
Census Tract 188, Orange County, Florida: Summa...                                           1637   
Census Tract 189, Orange County, Florida: Summa...                                           3264   
Census Tract 9900, Orange County, Florida: Summ...                                              0   

                                                                                                     \
                                                   Estimate!!RELATIONSHIP!!Population in households   
index                                                                                                 
Census Tract 102, Orange County, Florida: Summa...                                             4694   
Census Tract 103, Orange County, Florida: Summa...                                             2492   
Census Tract 104, Orange County, Florida: Summa...                                             1597   
Census Tract 105, Orange County, Florida: Summa...                                             1097   
Census Tract 108.02, Orange County, Florida: Su...                                             2777   
...                                                                                             ...   
Census Tract 185, Orange County, Florida: Summa...                                             3630   
Census Tract 187, Orange County, Florida: Summa...                                             5136   
Census Tract 188, Orange County, Florida: Summa...                                             3111   
Census Tract 189, Orange County, Florida: Summa...                                             6345   
Census Tract 9900, Orange County, Florida: Summ...                                                0   

                                                                                                                 \
                                                   Percent!!RELATIONSHIP!!Population in households!!Householder   
index                                                                                                             
Census Tract 102, Orange County, Florida: Summa...                                               62.9             
Census Tract 103, Orange County, Florida: Summa...                                               69.6             
Census Tract 104, Orange County, Florida: Summa...                                               30.2             
Census Tract 105, Orange County, Florida: Summa...                                               43.4             
Census Tract 108.02, Orange County, Florida: Su...                                               57.7             
...                                                                                               ...             
Census Tract 185, Orange County, Florida: Summa...                                               40.3             
Census Tract 187, Orange County, Florida: Summa...                                   

In [29]:
#Separate side by side for individual variables 
df_final = df_all.swaplevel(axis='columns')[sort2019.columns[1:]]
#Put these columns side by side for each variable 
df_final

Estimate!!RELATIONSHIP!!Population in households  \
                                                                                               2014   
index                                                                                                 
Census Tract 102, Orange County, Florida: Summa...                                             4694   
Census Tract 103, Orange County, Florida: Summa...                                             2492   
Census Tract 104, Orange County, Florida: Summa...                                             1597   
Census Tract 105, Orange County, Florida: Summa...                                             1097   
Census Tract 108.02, Orange County, Florida: Su...                                             2777   
...                                                                                             ...   
Census Tract 185, Orange County, Florida: Summa...                                             3630   
Census Tract 187, Orange County, Florida: Summa...                                             5136   
Census Tract 188, Orange County, Florida: Summa...                                             3111   
Census Tract 189, Orange County, Florida: Summa...                                             6345   
Census Tract 9900, Orange County, Florida: Summ...                                                0   

                                                          \
                                                    2019   
index                                                      
Census Tract 102, Orange County, Florida: Summa...  4935   
Census Tract 103, Orange County, Florida: Summa...  2279   
Census Tract 104, Orange County, Florida: Summa...  1127   
Census Tract 105, Orange County, Florida: Summa...  1218   
Census Tract 108.02, Orange County, Florida: Su...  3294   
...                                                  ...   
Census Tract 185, Orange County, Florida: Summa...  2864   
Census Tract 187, Orange County, Florida: Summa...  5381   
Census Tract 188, Orange County, Florida: Summa...  4577   
Census Tract 189, Orange County, Florida: Summa...  6565   
Census Tract 9900, Orange County, Florida: Summ...     0   

                                                   Percent!!RELATIONSHIP!!Population in households!!Householder  \
                                                                                                           2014   
index                                                                                                             
Census Tract 102, Orange County, Florida: Summa...                                               62.9             
Census Tract 103, Orange County, Florida: Summa...                                               69.6             
Census Tract 104, Orange County, Florida: Summa...                                               30.2             
Census Tract 105, Orange County, Florida: Summa...                                               43.4             
Census Tract 108.02, Orange County, Florida: Su...                                               57.7             
...                                                                                               ...             
Census Tract 185, Orange County, Florida: Summa...                                               40.3             
Census Tract 187, Orange County, Florida: Summa...                                               36.9             
Census Tract 188, Orange County, Florida: Summa...                                               52.6             
Census Tract 189, Orange County, Florida: Summa...                                               51.4             
Census Tract 9900, Orange County, Florida: Summ...                                       -666666666.0             

                                                                 \
                                                           2019   
index                 

In [30]:
dfh = df_final[['Estimate!!RELATIONSHIP!!Population in households!!Spouse']].head(2)
dfh

Estimate!!RELATIONSHIP!!Population in households!!Spouse  \
                                                                                                       2014   
index                                                                                                         
Census Tract 102, Orange County, Florida: Summa...                                                658         
Census Tract 103, Orange County, Florida: Summa...                                                202         

                                                         
                                                   2019  
index                                                    
Census Tract 102, Orange County, Florida: Summa...  839  
Census Tract 103, Orange County, Florida: Summa...  193

In [31]:
#Create a dataframe that shows the difference between years for each variable. We can then discover what census tracts and what variables have changed
#Iteration code that will run a subtraction script for each pair of data on each row for each variable. 
#Iteration code currently works for one piece of data 
# Create DataFrame to fill with combinations
comparison_df = pd.DataFrame(index=dfh.index)
# Create combinations of columns
for a, b in iter.combinations(dfh.columns, 2):
    # Subtract column combinations
    comparison_df['{}_{}'.format(a, b)] = ((dfh[a] - dfh[b])/dfh[a])*100

#print (Combination DataFrame)
print(comparison_df)

comparison_df

                                                   ('Estimate!!RELATIONSHIP!!Population in households!!Spouse', '2014')_('Estimate!!RELATIONSHIP!!Population in households!!Spouse', '2019')
index                                                                                                                                                                                       
Census Tract 102, Orange County, Florida: Summa...                                         -27.507599                                                                                       
Census Tract 103, Orange County, Florida: Summa...                                           4.455446                                                                                       


,"('Estimate!!RELATIONSHIP!!Population in households!!Spouse', '2014')_('Estimate!!RELATIONSHIP!!Population in households!!Spouse', '2019')"
index,
"Census Tract 102, Orange County, Florida: Summary level: 140, state:12> county:095> tract:010200",-27.507599
"Census Tract 103, Orange County, Florida: Summary level: 140, state:12> county:095> tract:010300",4.455446


In [32]:
#Iterating through a df this large does not work - too slow - need to find a more efficient way
comparison_df = pd.DataFrame(index=df_final.index)
# Create combinations of columns
eps = 1e-9
for a, b in iter.combinations(df_final.columns, 2):
    # Subtract column combinations
    comparison_df['{}_{}'.format(a, b)] = ((df_final[a] - df_final[b])/(df_final[a]+eps))*100

#print (Combination DataFrame)
print(comparison_df)

comparison_df

KeyboardInterrupt: 

524

,index,Estimate!!HOUSEHOLDS BY TYPE!!Total households,Estimate!!RELATIONSHIP!!Population in households,Percent!!RELATIONSHIP!!Population in households!!Householder,Estimate!!RELATIONSHIP!!Population in households!!Spouse,Percent!!RELATIONSHIP!!Population in households!!Spouse,Estimate!!RELATIONSHIP!!Population in households!!Child,Percent!!RELATIONSHIP!!Population in households!!Child,Estimate!!RELATIONSHIP!!Population in households!!Other relatives,Percent!!RELATIONSHIP!!Population in households!!Other relatives,...,Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Not Hispanic or Latino!!Native Hawaiian and Other Pacific Islander alone,Percent!!HISPANIC OR LATINO AND RACE!!Total population!!Not Hispanic or Latino!!Native Hawaiian and Other Pacific Islander alone,Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Not Hispanic or Latino!!Some other race alone,Percent!!HISPANIC OR LATINO AND RACE!!Total population!!Not Hispanic or Latino!!Some other race alone,Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Not Hispanic or Latino!!Two or more races,Percent!!HISPANIC OR LATINO AND RACE!!Total population!!Not Hispanic or Latino!!Two or more races,Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Not Hispanic or Latino!!Two or more races!!Two races including Some other race,Percent!!HISPANIC OR LATINO AND RACE!!Total population!!Not Hispanic or Latino!!Two or more races!!Two races including Some other race,"Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Not Hispanic or Latino!!Two or more races!!Two races excluding Some other race, and Three or more races","Percent!!HISPANIC OR LATINO AND RACE!!Total population!!Not Hispanic or Latino!!Two or more races!!Two races excluding Some other race, and Three or more races"
85,"Census Tract 102, Orange County, Florida: Summ...",2954,4694,62.9,658,14.0,161,3.4,113,2.4,...,74,1.5,0,0.0,72,1.5,38,0.8,34,0.7
22,"Census Tract 103, Orange County, Florida: Summ...",1734,2492,69.6,202,8.1,277,11.1,31,1.2,...,0,0.0,0,0.0,71,2.7,0,0.0,71,2.7
86,"Census Tract 104, Orange County, Florida: Summ...",483,1597,30.2,50,3.1,637,39.9,325,20.4,...,0,0.0,0,0.0,20,1.3,0,0.0,20,1.3
136,"Census Tract 105, Orange County, Florida: Summ...",476,1097,43.4,28,2.6,246,22.4,220,20.1,...,0,0.0,0,0.0,48,4.4,0,0.0,48,4.4
42,"Census Tract 108.02, Orange County, Florida: S...",1603,2777,57.7,327,11.8,428,15.4,101,3.6,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,"Census Tract 185, Orange County, Florida: Summ...",1464,3630,40.3,248,6.8,820,22.6,526,14.5,...,0,0.0,6,0.1,87,2.1,0,0.0,87,2.1
69,"Census Tract 187, Orange County, Florida: Summ...",1893,5136,36.9,703,13.7,1960,38.2,348,6.8,...,0,0.0,0,0.0,21,0.4,0,0.0,21,0.4
70,"Census Tract 188, Orange County, Florida: Summ...",1637,3111,52.6,376,12.1,438,14.1,122,3.9,...,0,0.0,13,0.4,53,1.7,22,0.7,31,1.0
71,"Census Tract 189, Orange County, Florida: Summ...",3264,6345,51.4,522,8.2,1475,23.2,319,5.0,...,0,0.0,23,0.3,97,1.3,15,0.2,82,1.1


,index,Estimate!!HOUSEHOLDS BY TYPE!!Total households,Estimate!!RELATIONSHIP!!Population in households,Percent!!RELATIONSHIP!!Population in households!!Householder,Estimate!!RELATIONSHIP!!Population in households!!Spouse,Percent!!RELATIONSHIP!!Population in households!!Spouse,Estimate!!RELATIONSHIP!!Population in households!!Child,Percent!!RELATIONSHIP!!Population in households!!Child,Estimate!!RELATIONSHIP!!Population in households!!Other relatives,Percent!!RELATIONSHIP!!Population in households!!Other relatives,...,Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Not Hispanic or Latino!!Native Hawaiian and Other Pacific Islander alone,Percent!!HISPANIC OR LATINO AND RACE!!Total population!!Not Hispanic or Latino!!Native Hawaiian and Other Pacific Islander alone,Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Not Hispanic or Latino!!Some other race alone,Percent!!HISPANIC OR LATINO AND RACE!!Total population!!Not Hispanic or Latino!!Some other race alone,Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Not Hispanic or Latino!!Two or more races,Percent!!HISPANIC OR LATINO AND RACE!!Total population!!Not Hispanic or Latino!!Two or more races,Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Not Hispanic or Latino!!Two or more races!!Two races including Some other race,Percent!!HISPANIC OR LATINO AND RACE!!Total population!!Not Hispanic or Latino!!Two or more races!!Two races including Some other race,"Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Not Hispanic or Latino!!Two or more races!!Two races excluding Some other race, and Three or more races","Percent!!HISPANIC OR LATINO AND RACE!!Total population!!Not Hispanic or Latino!!Two or more races!!Two races excluding Some other race, and Three or more races"
120,"Census Tract 102, Orange County, Florida: Summ...",2838,4935,57.5,839,17.0,503,10.2,38,0.8,...,0,0.0,21,0.4,55,1.1,0,0.0,55,1.1
93,"Census Tract 103, Orange County, Florida: Summ...",1539,2279,67.5,193,8.5,166,7.3,20,0.9,...,0,0.0,0,0.0,49,2.0,0,0.0,49,2.0
121,"Census Tract 104, Orange County, Florida: Summ...",461,1127,40.9,73,6.5,424,37.6,107,9.5,...,0,0.0,0,0.0,26,2.3,0,0.0,26,2.3
22,"Census Tract 105, Orange County, Florida: Summ...",493,1218,40.5,39,3.2,390,32.0,132,10.8,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
83,"Census Tract 108.02, Orange County, Florida: S...",1482,3294,45.0,571,17.3,552,16.8,74,2.2,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32,"Census Tract 185, Orange County, Florida: Summ...",1092,2864,38.1,274,9.6,820,28.6,271,9.5,...,0,0.0,49,1.5,124,3.8,0,0.0,124,3.8
33,"Census Tract 187, Orange County, Florida: Summ...",1810,5381,33.6,469,8.7,2299,42.7,502,9.3,...,0,0.0,0,0.0,62,1.1,0,0.0,62,1.1
34,"Census Tract 188, Orange County, Florida: Summ...",2377,4577,51.9,447,9.8,482,10.5,193,4.2,...,0,0.0,12,0.3,77,1.7,0,0.0,77,1.7
35,"Census Tract 189, Orange County, Florida: Summ...",4129,6565,62.9,491,7.5,808,12.3,271,4.1,...,0,0.0,17,0.2,104,1.5,12,0.2,92,1.3


207

207

Index(['Estimate!!HOUSEHOLDS BY TYPE!!Total households',
       'Estimate!!RELATIONSHIP!!Population in households',
       'Percent!!RELATIONSHIP!!Population in households!!Householder',
       'Estimate!!RELATIONSHIP!!Population in households!!Spouse',
       'Percent!!RELATIONSHIP!!Population in households!!Spouse',
       'Estimate!!RELATIONSHIP!!Population in households!!Child',
       'Percent!!RELATIONSHIP!!Population in households!!Child',
       'Estimate!!RELATIONSHIP!!Population in households!!Other relatives',
       'Percent!!RELATIONSHIP!!Population in households!!Other relatives',
       'Estimate!!MARITAL STATUS!!Males 15 years and over',
       ...
       'Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Not Hispanic or Latino!!Native Hawaiian and Other Pacific Islander alone',
       'Percent!!HISPANIC OR LATINO AND RACE!!Total population!!Not Hispanic or Latino!!Native Hawaiian and Other Pacific Islander alone',
       'Estimate!!HISPANIC OR LATINO AND RACE!!T

Estimate!!HOUSEHOLDS BY TYPE!!Total households                                                                                                                      int64
Estimate!!RELATIONSHIP!!Population in households                                                                                                                    int64
Percent!!RELATIONSHIP!!Population in households!!Householder                                                                                                        int64
Estimate!!RELATIONSHIP!!Population in households!!Spouse                                                                                                            int64
Percent!!RELATIONSHIP!!Population in households!!Spouse                                                                                                             int64
                                                                                                                                                      

Percent!!HOUSE HEATING FUEL!!Occupied housing units!!Bottled, tank, or LP gas                                                       1.0
Percent!!ANCESTRY!!Total population!!Slovak                                                                                         1.0
Percent!!HOUSE HEATING FUEL!!Occupied housing units!!Coal or coke                                                                   1.0
Percent!!HOUSE HEATING FUEL!!Occupied housing units!!Wood                                                                           1.0
Percent!!UNITS IN STRUCTURE!!Total housing units!!Boat, RV, van, etc.                                                               1.0
Percent!!HISPANIC OR LATINO AND RACE!!Total population!!Not Hispanic or Latino!!Native Hawaiian and Other Pacific Islander alone    1.0
Percent!!ANCESTRY!!Total population!!Czech                                                                                          1.0
Percent!!EMPLOYMENT STATUS!!Population 16 years 

In [21]:
sort_mat.head(10)

Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Not Hispanic or Latino!!American Indian and Alaska Native alone                         -0.057302
Estimate!!CLASS OF WORKER!!Civilian employed population 16 years and over!!Unpaid family workers                                                 -0.034690
Estimate!!HOUSE HEATING FUEL!!Occupied housing units!!Other fuel                                                                                 -0.017607
Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Not Hispanic or Latino!!Native Hawaiian and Other Pacific Islander alone                -0.013846
Estimate!!HOUSE HEATING FUEL!!Occupied housing units!!Coal or coke                                                                               -0.006747
Estimate!!SELECTED CHARACTERISTICS!!Occupied housing units!!Lacking complete plumbing facilities                                                  0.008021
Estimate!!HOUSE HEATING FUEL!!Occupied housing units!!Wood            

This correlation has been completed to demonstrate the differences between 2014 and 2019 Orange county florida - ACS data. First presented in the negative correlation, that has variables that have shown a significant increase within the 5 years. There are over 100 rows that demonstrate a positive correlation of +1. There are 10 that are listed below. 

Percent!!HOUSE HEATING FUEL!!Occupied housing units!!Bottled, tank, or LP gas                                                       1.0
Percent!!ANCESTRY!!Total population!!Slovak                                                                                         1.0
Percent!!HOUSE HEATING FUEL!!Occupied housing units!!Coal or coke                                                                   1.0
Percent!!HOUSE HEATING FUEL!!Occupied housing units!!Wood                                                                           1.0
Percent!!UNITS IN STRUCTURE!!Total housing units!!Boat, RV, van, etc.                                                               1.0
Percent!!HISPANIC OR LATINO AND RACE!!Total population!!Not Hispanic or Latino!!Native Hawaiian and Other Pacific Islander alone    1.0
Percent!!ANCESTRY!!Total population!!Czech                                                                                          1.0
Percent!!EMPLOYMENT STATUS!!Population 16 years 

,Estimate!!HOUSEHOLDS BY TYPE!!Total households,Estimate!!RELATIONSHIP!!Population in households,Percent!!RELATIONSHIP!!Population in households!!Householder,Estimate!!RELATIONSHIP!!Population in households!!Spouse,Percent!!RELATIONSHIP!!Population in households!!Spouse,Estimate!!RELATIONSHIP!!Population in households!!Child,Percent!!RELATIONSHIP!!Population in households!!Child,Estimate!!RELATIONSHIP!!Population in households!!Other relatives,Percent!!RELATIONSHIP!!Population in households!!Other relatives,Estimate!!MARITAL STATUS!!Males 15 years and over,...,Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Not Hispanic or Latino!!Native Hawaiian and Other Pacific Islander alone,Percent!!HISPANIC OR LATINO AND RACE!!Total population!!Not Hispanic or Latino!!Native Hawaiian and Other Pacific Islander alone,Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Not Hispanic or Latino!!Some other race alone,Percent!!HISPANIC OR LATINO AND RACE!!Total population!!Not Hispanic or Latino!!Some other race alone,Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Not Hispanic or Latino!!Two or more races,Percent!!HISPANIC OR LATINO AND RACE!!Total population!!Not Hispanic or Latino!!Two or more races,Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Not Hispanic or Latino!!Two or more races!!Two races including Some other race,Percent!!HISPANIC OR LATINO AND RACE!!Total population!!Not Hispanic or Latino!!Two or more races!!Two races including Some other race,"Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Not Hispanic or Latino!!Two or more races!!Two races excluding Some other race, and Three or more races","Percent!!HISPANIC OR LATINO AND RACE!!Total population!!Not Hispanic or Latino!!Two or more races!!Two races excluding Some other race, and Three or more races"
index,,,,,,,,,,,,,,,,,,,,,
"Census Tract 102, Orange County, Florida: Summary level: 140, state:12> county:095> tract:010200",2838,4935,57,839,17,503,10,38,0,2629,...,0,0,21,0,55,1,0,0,55,1
"Census Tract 103, Orange County, Florida: Summary level: 140, state:12> county:095> tract:010300",1539,2279,67,193,8,166,7,20,0,1002,...,0,0,0,0,49,2,0,0,49,2
"Census Tract 104, Orange County, Florida: Summary level: 140, state:12> county:095> tract:010400",461,1127,40,73,6,424,37,107,9,394,...,0,0,0,0,26,2,0,0,26,2
"Census Tract 105, Orange County, Florida: Summary level: 140, state:12> county:095> tract:010500",493,1218,40,39,3,390,32,132,10,395,...,0,0,0,0,0,0,0,0,0,0
"Census Tract 108.02, Orange County, Florida: Summary level: 140, state:12> county:095> tract:010802",1482,3294,45,571,17,552,16,74,2,1538,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Census Tract 185, Orange County, Florida: Summary level: 140, state:12> county:095> tract:018500",1092,2864,38,274,9,820,28,271,9,1455,...,0,0,49,1,124,3,0,0,124,3
"Census Tract 187, Orange County, Florida: Summary level: 140, state:12> county:095> tract:018700",1810,5381,33,469,8,2299,42,502,9,1829,...,0,0,0,0,62,1,0,0,62,1
"Census Tract 188, Orange County, Florida: Summary level: 140, state:12> county:095> tract:018800",2377,4577,51,447,9,482,10,193,4,2353,...,0,0,12,0,77,1,0,0,77,1


In [25]:
#Concat dfs so that we can see side by side 

df_all = pd.concat([sort2014.set_index('index'), sort2019.set_index('index')], 
                   axis='columns', keys=['First', 'Second'])
df_all

KeyError: "None of ['index'] are in the columns"